In [15]:
import timeit
start = timeit.default_timer()

import os
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'
import sys
print(sys.version)

3.10.9 (main, Mar  1 2023, 18:23:06) [GCC 11.2.0]


In [16]:
import folium
from folium.plugins import MarkerCluster
import pyarrow as pa
import geohash_tools as gh
import pyspark as ps

In [17]:
sp = spark.read.load('hdfs://orion11:14001/nam/2019/01/05/*',
                     format='csv',
                     sep='\t',
                     inferSchema='true',
                     header='true')

schema = sp.schema

sp = spark.read.schema(schema).load('hdfs://orion11:14001/nam/201*/*/*/*',
                     format='csv',
                     sep='\t',
                     header='true')
sp.printSchema()

root
 |-- 1_time: long (nullable = true)
 |-- 2_lat: double (nullable = true)
 |-- 3_lon: double (nullable = true)
 |-- albedo_surface: double (nullable = true)
 |-- precipitable_water_entire_atmosphere_single_layer: double (nullable = true)
 |-- pressure_maximum_wind: double (nullable = true)
 |-- pressure_surface: double (nullable = true)
 |-- pressure_tropopause: double (nullable = true)
 |-- relative_humidity_zerodegc_isotherm: double (nullable = true)
 |-- snow_depth_surface: double (nullable = true)
 |-- temperature_surface: double (nullable = true)
 |-- temperature_tropopause: double (nullable = true)
 |-- total_cloud_cover_entire_atmosphere_single_layer: double (nullable = true)
 |-- total_precipitation_surface_3_hour_accumulation: double (nullable = true)
 |-- vegetation_surface: double (nullable = true)
 |-- visibility_surface: double (nullable = true)
 |-- wilting_point_surface: double (nullable = true)
 |-- wind_speed_gust_surface: double (nullable = true)
 |-- _c18: string

In [18]:
sp = sp.select(['2_lat','3_lon','visibility_surface'])

In [ ]:
fog_sanctuaries = sp.rdd \
    .map(lambda x: (gh.encode(x[0],x[1]), (x[2], 1))) \
    .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])) \
    .map(lambda x: (x[0], x[1][0]/x[1][1])) \
    .sortBy(lambda x: x[1], ascending=False) \
    .take(40)

In [ ]:
m = folium.Map(location=[41,-98.58], zoom_start=3)
mc = MarkerCluster().add_to(m)

for fs in fog_sanctuaries:
    folium.Marker(location=list(gh.decode(fs[0])), popup=str(fs[1])).add_to(mc)

m

In [ ]:
print(timeit.default_timer()-start)